In [1]:
import skimage.measure as msr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
from PIL import Image, ImageCms

import glob
import os
from pathlib import Path
import imageio
import cv2

In [2]:
import sys
sys.path.append("..")

In [3]:
# Importing class from wmark.py
from wmark import WaterMark

In [4]:
# Source Directory
srcFolder = 'TestSet/'
# Source Path
srcPth = Path(srcFolder).resolve()

# Destination Directory
# dstFolder = Path(srcFolder+'/marked/').resolve()
# Creating folder for marked images
# Path(dstFolder).mkdir(exist_ok=True)
# Destination Path
# dstPath = Path(dstFolder).resolve()

In [5]:
# Creating empty DataFrame that will fill up after the loop
metricDataframe = pd.DataFrame()
for i in range(1, 2):
    
    # Generating random seed as integer up to 10000
    randomNums = np.random.randint(10000, size=(1))
    randomNum = randomNums[0]
    print(randomNum)

    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    results = np.zeros([numImages, 10])
    counter = 0
        

    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgMarked.shape[0]
        # Height
        ly = imgMarked.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgMarked[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgMarked, (width, height))

        
        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgMarked.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgMarked, M, (cols, rows))    

        
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgMarked.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgMarked, M, (cols, rows))
       
        
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgMarked, -1, kernel)
       
        
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgMarked, (kernelX, kernelY), cv2.BORDER_DEFAULT)
  
        
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgMarked.size)
        gauss = gauss.reshape(
            imgMarked.shape[0], imgMarked.shape[1], imgMarked.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgMarked, gauss)
      
        
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        results[counter, 0] = metricOriginal
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        results[counter, 1] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        results[counter, 2] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        results[counter, 3] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        results[counter, 4] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        results[counter, 5] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        results[counter, 6] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        results[counter, 7] = metricNoised
        
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        results[counter, 8] = metricMarked
        
        metricImpactFactor = ImpactFactor
        results[counter, 9] = metricImpactFactor
        
        counter += 1
        
        
        metricValues = ["Original", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised", "GCR / Marked", "Impact Factor"]
        metricSeeds = [f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}"]
        
        finalMetric = pd.DataFrame(results, columns = [metricSeeds, metricValues])
        
        
    metricDataframe = pd.concat([metricDataframe, finalMetric], axis = 1)

2399


/Users/zgebac/miniconda3/envs/img-proc/lib/python3.7/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
# Creating empty DataFrame that will fill up after the loop
gcrMetricDataframe = pd.DataFrame()

for i in range(1, 2):
    
    # Generating random seed as integer up to 10000
#     randomNums = np.random.randint(10000, size=(1))
#     randomNum = randomNums[0]
#     print(randomNum)

    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    gcrResults = np.zeros([numImages, 10])
    counter = 0
        

    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        # GCR Masking
        imgGCR = WaterMark.gcrMasking(imgOriginal, imgMarked)
        
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgGCR.shape[0]
        # Height
        ly = imgGCR.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgGCR[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgGCR, (width, height))

        
        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgGCR.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgGCR, M, (cols, rows))    

        
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgGCR.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgGCR, M, (cols, rows))
       
        
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgGCR, -1, kernel)
       
        
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgGCR, (kernelX, kernelY), cv2.BORDER_DEFAULT)
  
        
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgGCR.size)
        gauss = gauss.reshape(
            imgGCR.shape[0], imgGCR.shape[1], imgGCR.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgGCR, gauss)
      
        
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        gcrResults[counter, 0] = metricOriginal
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        gcrResults[counter, 1] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        gcrResults[counter, 2] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        gcrResults[counter, 3] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        gcrResults[counter, 4] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        gcrResults[counter, 5] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        gcrResults[counter, 6] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        gcrResults[counter, 7] = metricNoised
        
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        gcrResults[counter, 8] = metricMarked
        
        metricImpactFactor = ImpactFactor
        gcrResults[counter, 9] = metricImpactFactor
        
        counter += 1
        
        
        gcrMetricValues = ["Original", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised", "GCR / Marked", "Impact Factor"]
        gcrMetricSeeds = [f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}"]
        
        
        gcrFinalMetric = pd.DataFrame(gcrResults, columns = [gcrMetricSeeds, gcrMetricValues])
        
        
    gcrMetricDataframe = pd.concat([gcrMetricDataframe, gcrFinalMetric], axis = 1)

In [9]:
# Show Pandas DataFrame
metricDataframe

Marked 2399                                                                  \
     Original   Cropped    Scaled   Rotated   Affined 2D Convoluted   Blurred   
0    0.211486  0.530770  0.177947  0.642814  0.281666      0.684571  0.686065   
1    0.227935  0.456407  0.193078  0.605912  0.263425      0.738682  0.740017   
2    0.195372  0.711112  0.169804  0.731704  0.249018      0.866920  0.868743   
3    0.281601  0.581868  0.223513  0.653514  0.269888      0.753030  0.753459   
4    0.240805  0.736481  0.206132  0.776354  0.268373      0.923665  0.924629   

                                        
     Noised GCR / Marked Impact Factor  
0  0.711712     0.737481    1303.21000  
1  0.742422     0.752977     952.67909  
2  0.887099     0.917873    1303.21000  
3  0.716225     0.762968    1303.21000  
4  0.931805     0.943339    1114.24455

In [10]:
# Show Pandas DataFrame
gcrMetricDataframe

Marked 2399                                                                  \
     Original   Cropped    Scaled   Rotated   Affined 2D Convoluted   Blurred   
0    0.211486  0.373215  0.187069  0.410781  0.296005      0.307348  0.311898   
1    0.227935  0.294031  0.185444  0.476644  0.277304      0.566399  0.569239   
2    0.195372  0.473917  0.162030  0.545730  0.256660      0.493121  0.500041   
3    0.281601  0.540481  0.212121  0.610821  0.268387      0.672679  0.674169   
4    0.240805  0.533608  0.200645  0.708403  0.266831      0.796454  0.800714   

                                        
     Noised GCR / Marked Impact Factor  
0  0.397957     0.737481    1303.21000  
1  0.581053     0.752977     952.67909  
2  0.530039     0.917873    1303.21000  
3  0.644674     0.762968    1303.21000  
4  0.819561     0.943339    1114.24455

In [ ]:
gcrMetricDataframe.boxplot(rot=90)

In [11]:
describeDataframe = metricDataframe.describe()

In [12]:
gcrdescribeDataframe = gcrMetricDataframe.describe()

In [14]:
compareDataframe = describeDataframe.join(gcrdescribeDataframe)

ValueError: columns overlap but no suffix specified: MultiIndex(levels=[['Marked 2399'], ['2D Convoluted', 'Affined', 'Blurred', 'Cropped', 'GCR / Marked', 'Impact Factor', 'Noised', 'Original', 'Rotated', 'Scaled']],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [7, 3, 9, 8, 1, 0, 2, 6, 4, 5]])

In [ ]:
comparemeanDataframe = compareDataframe.iloc[1]

In [ ]:
comparemeanDataframeUnstacked = comparemeanDataframe.unstack(level = 0)

In [ ]:
comparemeanDataframeUnstacked

In [ ]:
# Colors: https://github.com/matplotlib/matplotlib/blob/38be7aeaaac3691560aeadafe46722dda427ef47/lib/matplotlib/mpl-data/stylelib/fivethirtyeight.mplstyle

plt.style.use('fivethirtyeight')

# Creating graph
fig, ax1 = plt.subplots(figsize=(12,8))

# Setting X-axis and first Y-axis
ax1.set_xlabel('Impact Factor')
ax1.set_ylabel('PSNR', color = '#008fd5')

# Creating a second Y-axis that shares the same X-axis
ax2 = ax1.twinx()  
ax2.set_ylabel('SSIM', color = '#fc4f30')

# Show grid
ax1.grid(True)
ax2.grid(True)

# Setting different colors for each variable
ax1.plot(comparemeanDataframeUnstacked['2982'], color = '#008fd5')
ax2.plot(comparemeanDataframeUnstacked['9721'], color = '#fc4f30')

In [ ]:
stackedDataframe.plot.hist(legend = True)